In [1]:
import pandas as pd

In [2]:
injected = pd.read_csv('injection_prediction.csv')
uninjected = pd.read_csv('no_injection_prediction.csv')

In [3]:
injected = injected.drop_duplicates()
uninjected = uninjected.drop_duplicates()

In [5]:
full = pd.concat([injected, uninjected], ignore_index = True)

In [7]:
full = full.drop_duplicates(subset = ['ci', 'intensity', 'type', 'localization'])

In [9]:
# generating training heatmap
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.use("Agg")

req = full[full['anomaly'] == True]
req = req[['intensity', 'type', 'ci']]
req['intensity'] = req['intensity']/16
req['ci_category'] = req['ci'].round(decimals=3)
bins = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, float('inf')]
labels = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9']
req['timeline'] = pd.cut(req['ci_category'], bins = bins, labels=labels).astype('string').fillna('T0')

req = req.drop(columns = ['ci', 'ci_category'])
req['timeline'] = pd.Categorical(req['timeline'], categories=['T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10'], ordered=True)
heatmap_data = req.pivot_table(index='type', columns='timeline', values = 'intensity', aggfunc = 'mean')
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, annot =True, cmap = 'coolwarm')
plt.xlabel('Intensity')
plt.ylabel('Type')
plt.savefig('eval_distribution.png')
plt.show()

/tmp/ipykernel_333101/3381592889.py:23: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [ ]:
#FPR and TPR
